In [55]:
from gezi.common import *
sys.path.append('..')
from src.postprocess import *
from src.config import *
from src.eval import *
from src.ensemble_conf import mns, get_weight

In [53]:
gezi.init_flags()
df = pd.read_csv(f'{FLAGS.root}/train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [56]:
FOLDS = 5
xs = []
ensemblers = [gezi.Ensembler(need_sort=True, includes=['bert_proba']) for _ in range(FOLDS)]
for model in mns:
  for fold in range(FOLDS):
    root = f'../working/offline/10/{fold}/{model}'
    x = gezi.load(f'{root}/valid.pkl')
    x['fold'] = np.asarray([fold] * len(x['id']))
    x['bert_proba'] = normalize(x['pred'])
    # xs.append(x)
    ensemblers[fold].add(x, get_weight(model))
xs = [ensembler.finalize() for ensembler in ensemblers]
for x in xs:
  x['bert_proba'] = normalize(x['bert_proba'])
x = gezi.merge_array_dicts(xs)
x

{'id': array(['00068e3dfd5515df', '000c8495ef1763e1', '0019b063a0fe1dc8', ...,
        'ffb6e883de7d9d67', 'fff0b4d9fca7fae3', 'fff8d1ccce8e20ec'],
       dtype='<U16'),
 'label': array([0.25, 0.  , 0.  , ..., 0.5 , 0.25, 0.  ]),
 'anchor': array(['conductor particles', 'page file', 'predetermined acceleration',
        ..., 'water propulsion device',
        'high gradient magnetic separators', 'perform working operations'],
       dtype='<U38'),
 'target': array(['resin cores', 'hidden camera', 'predetermined policy', ...,
        'propulsion unit', 'magnetic equipment',
        'psychology of working framework'], dtype='<U98'),
 'text': array(['conductor particles[SEP]resin cores[SEP]PERFORMING OPERATIONS; TRANSPORTING. PHYSICAL OR CHEMICAL PROCESSES OR APPARATUS IN GENERAL[SEP]metal; conducting particles; gold tooth; transparent spheres particles; ballpen ink; sources; exchange membranes; electro conductive powders; source membranes; conductor fine particles; source electrodes; con

In [57]:
x['score'] = x['label']
# x['bert_proba'] = normalize(x['bert_proba'])

In [58]:
del x['text']
del x['pred']
del x['label']

In [59]:
oof = pd.DataFrame(x)

In [60]:
calc_metric(oof.score, oof.bert_proba)

0.8682403177522899

In [61]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from lightgbm.sklearn import LGBMRegressor
from sklearn.linear_model import BayesianRidge
import time


def run_stacking(train_data, test_data):
    '''
    Input:
        train_data: [id, anchor, target, context, score, bert_proba, fold]
        test_data: [id, anchor, target, context, bert_proba]
    
    Return:
        X: ['id', 'score', 'bert_proba', 'tree_proba', 'stacking_proba']
        X_test: ['id', 'bert_proba', 'tree_proba', 'stacking_proba']
    '''
    
    FOLD = train_data['fold'].nunique()
    
    
    def edit_distance(x, y):
        len1 = len(x)
        len2 = len(y)
        dp = np.zeros((len1 + 1, len2 + 1))
        for i in range(len1 + 1):
            dp[i][0] = i
        for j in range(len2 + 1):
            dp[0][j] = j
        for i in range(1, len1 + 1):
            for j in range(1, len2 + 1):
                delta = 0 if x[i - 1] == y[j - 1] else 1
                dp[i][j] = min(dp[i - 1][j - 1] + delta, min(dp[i - 1][j] + 1, dp[i][j - 1] + 1))
        return dp[len1][len2]

    def common_prefix_len(x, y):
        min_len = min(len(x), len(y))
        res = 0
        for i in range(min_len):
            if x[i] != y[i]:
                break
            res += 1
        return res

    def common_suffix_len(x, y):
        min_len = min(len(x), len(y))
        res = 0
        for i in range(1, min_len + 1):
            if x[-i] != y[-i]:
                break
            res += 1
        return res

    def LCStr(x, y):
        len1 = len(x)
        len2 = len(y)
        record = [[0 for i in range(len2 + 1)] for j in range(len1 + 1)]
        maxLen, p = 0, 0
        for i in range(len1):
            for j in range(len2):
                if x[i] == y[j]:
                    record[i + 1][j + 1] = record[i][j] + 1
                    if record[i + 1][j + 1] > maxLen:
                        maxLen = record[i + 1][j + 1]
                        p = i + 1
        return x[p - maxLen : p]

    def head_common_index(x, y):
        for i, j in enumerate(y):
            if j == x[0]:
                return i
        return -1

    def combine(x, y):
        res = []
        for i in x:
            for j in y:
                res.append(i + '_' + j)
        return res
    
    def feat_eng_core(train_data, test_data):
        train_data['tag'] = 0
        test_data['tag'] = 1
        df = pd.concat([train_data, test_data], axis=0, ignore_index=True)

        df['context_code'] = df['context'].apply(lambda x: x[0])
        df['context_num'] = df['context'].apply(lambda x: x[1:]).astype('int16')
        
        for f in tqdm(['anchor', 'context']):
            df[f'{f}_count'] = df[f].map(df[f].value_counts())

        df['anchor_context_count'] = df.groupby(['anchor', 'context'])['anchor'].transform('count')
        df['anchor_in_context_count_prop'] = df['anchor_context_count'] / df['context_count']

        df['anchor_in_context_nunique'] = df.groupby('context')['anchor'].transform('nunique')
        df['anchor_in_context_nunique_count_ratio'] = df['anchor_in_context_nunique'] / df['context_count']

        df['context_in_anchor_nunique'] = df.groupby('anchor')['context'].transform('nunique')
        df['context_in_anchor_nunique_count_ratio'] = df['context_in_anchor_nunique'] / df['anchor_count']

        df['anchor_context_code_count'] = df.groupby(['anchor', 'context_code'])['anchor'].transform('count')
        df['anchor_in_context_code_nunique'] = df.groupby('context_code')['anchor'].transform('nunique')
        df['context_code_in_anchor_nunique'] = df.groupby('anchor')['context_code'].transform('nunique')
        
        for f in tqdm(['anchor', 'target']):
            df[f'{f}_char_list'] = df[f].apply(lambda x: list(x.replace(' ', '')))
            df[f'{f}_word_list'] = df[f].apply(lambda x: x.split(' '))

            df[f'{f}_char_set'] = df[f].apply(lambda x: set(x.replace(' ', '')))
            df[f'{f}_word_set'] = df[f].apply(lambda x: set(x.split(' ')))

            df[f'{f}_char_len'] = df[f'{f}_char_list'].apply(len)
            df[f'{f}_word_len'] = df[f'{f}_word_list'].apply(len)

        for f in tqdm(['char_len', 'word_len']):
            df[f'{f}_diff'] = abs(df[f'anchor_{f}'] - df[f'target_{f}'])
            df[f'{f}_diff_in_anchor_prop'] = df[f'{f}_diff'] / df[f'anchor_{f}']
            df[f'{f}_diff_in_target_prop'] = df[f'{f}_diff'] / df[f'target_{f}']
            df[f'{f}_ratio'] = df[f'anchor_{f}'] / df[f'target_{f}']

        for f in tqdm(['char', 'word']):
            df[f'{f}_inter'] = df[[f'anchor_{f}_set', f'target_{f}_set']].apply(
                lambda x: len(x[f'anchor_{f}_set'] & x[f'target_{f}_set']), axis=1
            )
            df[f'{f}_union'] = df[[f'anchor_{f}_set', f'target_{f}_set']].apply(
                lambda x: len(x[f'anchor_{f}_set'] | x[f'target_{f}_set']), axis=1
            )
            df[f'{f}_IoU'] = df[f'{f}_inter'] / df[f'{f}_union']
        
        df['char_edit_dist'] = df[['anchor_char_list', 'target_char_list']].apply(
            lambda x: edit_distance(x['anchor_char_list'], x['target_char_list']), axis=1)
        df['word_edit_dist'] = df[['anchor_word_list', 'target_word_list']].apply(
            lambda x: edit_distance(x['anchor_word_list'], x['target_word_list']), axis=1)
        
        for f in tqdm(['char', 'word']):
            df[f'{f}_common_prefix_len'] = df[[f'anchor_{f}_list', f'target_{f}_list']].apply(
                lambda x: common_prefix_len(x[f'anchor_{f}_list'], x[f'target_{f}_list']), axis=1)
            df[f'{f}_common_prefix'] = df[[f'anchor_{f}_list', f'{f}_common_prefix_len']].apply(
                lambda x: ' '.join(x[f'anchor_{f}_list'][:x[f'{f}_common_prefix_len']])
                          if x[f'{f}_common_prefix_len'] > 0 else '', axis=1
            )

            df[f'{f}_common_suffix_len'] = df[[f'anchor_{f}_list', f'target_{f}_list']].apply(
                lambda x: common_suffix_len(x[f'anchor_{f}_list'], x[f'target_{f}_list']), axis=1)
            df[f'{f}_common_suffix'] = df[[f'anchor_{f}_list', f'{f}_common_suffix_len']].apply(
                lambda x: ' '.join(x[f'anchor_{f}_list'][-x[f'{f}_common_suffix_len']:])
                          if x[f'{f}_common_suffix_len'] > 0 else '', axis=1
            )

            df[f'{f}_common_prefix_suffix_count'] = df.groupby(
                [f'{f}_common_prefix', f'{f}_common_suffix'])['anchor'].transform('count')

            df[f'{f}_LCStr'] = df[[f'anchor_{f}_list', f'target_{f}_list']].apply(
                lambda x: LCStr(x[f'anchor_{f}_list'], x[f'target_{f}_list']), axis=1)
            df[f'{f}_LCStr_len'] = df[f'{f}_LCStr'].apply(len)
            df[f'{f}_LCStr'] = df[f'{f}_LCStr'].apply(''.join)
            df[f'{f}_LCStr_count'] = df[f'{f}_LCStr'].map(df[f'{f}_LCStr'].value_counts())

            df[f'anchor_in_target_{f}_head_common_index'] = df[[f'anchor_{f}_list', f'target_{f}_list']].apply(
                lambda x: head_common_index(x[f'anchor_{f}_list'], x[f'target_{f}_list']), axis=1)
            df[f'target_in_anchor_{f}_head_common_index'] = df[[f'anchor_{f}_list', f'target_{f}_list']].apply(
                lambda x: head_common_index(x[f'target_{f}_list'], x[f'anchor_{f}_list']), axis=1)
        
        cate_cols = ['anchor', 'context', 'context_code']
        for f in tqdm(['anchor', 'target']):
            df[f'{f}_first_word'] = df[f'{f}_word_list'].apply(lambda x: x[0])
            df[f'{f}_last_word'] = df[f'{f}_word_list'].apply(lambda x: x[-1])

            df[f'{f}_first_word_count'] = df[f'{f}_first_word'].map(df[f'{f}_first_word'].value_counts())
            df[f'{f}_last_word_count'] = df[f'{f}_last_word'].map(df[f'{f}_last_word'].value_counts())

            cate_cols.extend([
                f'{f}_first_word',
                f'{f}_last_word'
            ])

        for f in tqdm(['word']):
            df[f'first_{f}_count'] = df.groupby([f'anchor_first_{f}', f'target_first_{f}'])['anchor'].transform('count')
            df[f'first_{f}_count_in_anchor_prop'] = df[f'first_{f}_count'] / df[f'anchor_first_{f}_count']
            df[f'first_{f}_count_in_target_prop'] = df[f'first_{f}_count'] / df[f'target_first_{f}_count']

            df[f'last_{f}_count'] = df.groupby([f'anchor_last_{f}', f'target_last_{f}'])['anchor'].transform('count')
            df[f'last_{f}_count_in_anchor_prop'] = df[f'last_{f}_count'] / df[f'anchor_last_{f}_count']
            df[f'last_{f}_count_in_target_prop'] = df[f'last_{f}_count'] / df[f'target_last_{f}_count']
        
        df['anchor_isin_target'] = df[['anchor', 'target']].apply(
            lambda x: x['anchor'] in x['target'], axis=1).astype('int8')
        df['target_isin_anchor'] = df[['anchor', 'target']].apply(
            lambda x: x['target'] in x['anchor'], axis=1).astype('int8')

        df['anchor_word_isin_target'] = df[['anchor_word_set', 'target_word_set']].apply(
            lambda x: x['anchor_word_set'].issubset(x['target_word_set']), axis=1).astype('int8')
        df['target_word_isin_anchor'] = df[['anchor_word_set', 'target_word_set']].apply(
            lambda x: x['target_word_set'].issubset(x['anchor_word_set']), axis=1).astype('int8')

        df['first_word_issame'] = (df['anchor_first_word'] == df['target_first_word']).astype('int8')
        df['last_word_issame'] = (df['anchor_last_word'] == df['target_last_word']).astype('int8')
        
        word_cnt_dict = {}
        lists = df['anchor_word_list'].values.tolist() + df['target_word_list'].values.tolist()
        for l in tqdm(lists):
            for w in l:
                if w not in word_cnt_dict.keys():
                    word_cnt_dict[w] = 0
                word_cnt_dict[w] += 1

        df['anchor_word_cnt_list'] = df['anchor_word_list'].apply(lambda x: [word_cnt_dict[c] for c in x])
        df['anchor_word_cnt_max'] = df['anchor_word_cnt_list'].apply(np.max)
        df['anchor_word_cnt_min'] = df['anchor_word_cnt_list'].apply(np.min)
        df['anchor_word_cnt_mean'] = df['anchor_word_cnt_list'].apply(np.mean)
        df['anchor_word_cnt_sum'] = df['anchor_word_cnt_list'].apply(np.sum)
        df['anchor_word_cnt_std'] = df['anchor_word_cnt_list'].apply(np.std)

        df['target_word_cnt_list'] = df['target_word_list'].apply(lambda x: [word_cnt_dict[c] for c in x])
        df['target_word_cnt_max'] = df['target_word_cnt_list'].apply(np.max)
        df['target_word_cnt_min'] = df['target_word_cnt_list'].apply(np.min)
        df['target_word_cnt_mean'] = df['target_word_cnt_list'].apply(np.mean)
        df['target_word_cnt_sum'] = df['target_word_cnt_list'].apply(np.sum)
        df['target_word_cnt_std'] = df['target_word_cnt_list'].apply(np.std)

        df['word_cnt_sum_ratio'] = df['anchor_word_cnt_sum'] / df['target_word_cnt_sum']

        del word_cnt_dict, lists
        
        df['word_combine_list'] = df[['anchor_word_list', 'target_word_list']].apply(
            lambda x: combine(x['anchor_word_list'], x['target_word_list']), axis=1)

        word_combine_cnt_dict = {}
        lists = df['word_combine_list'].values.tolist()
        for l in tqdm(lists):
            for w in l:
                if w not in word_combine_cnt_dict.keys():
                    word_combine_cnt_dict[w] = 0
                word_combine_cnt_dict[w] += 1

        df['word_combine_cnt_list'] = df['word_combine_list'].apply(lambda x: [word_combine_cnt_dict[c] for c in x])
        df['word_combine_cnt_max'] = df['word_combine_cnt_list'].apply(np.max)
        df['word_combine_cnt_min'] = df['word_combine_cnt_list'].apply(np.min)
        df['word_combine_cnt_median'] = df['word_combine_cnt_list'].apply(np.median)
        df['word_combine_cnt_mean'] = df['word_combine_cnt_list'].apply(np.mean)
        df['word_combine_cnt_sum'] = df['word_combine_cnt_list'].apply(np.sum)
        df['word_combine_cnt_std'] = df['word_combine_cnt_list'].apply(np.std)

        del word_combine_cnt_dict, lists
        
        for f in tqdm([
            'anchor_char_list', 'anchor_word_list', 'target_char_list', 'target_word_list',
            'anchor_char_set', 'anchor_word_set', 'target_char_set', 'target_word_set',
            'anchor_word_cnt_list', 'target_word_cnt_list', 'word_combine_list', 'word_combine_cnt_list',
            'char_common_prefix', 'char_common_suffix', 'char_LCStr',
            'word_common_prefix', 'word_common_suffix', 'word_LCStr',
        ]):
            del df[f]
        
        for f in tqdm(cate_cols):
            uniq = df[df[f].notna()][f].unique()
            df[f] = df[f].map(dict(zip(uniq, range(len(uniq)))))
        
        X = df[df['tag'] == 0].reset_index(drop=True)
        X_test = df[df['tag'] == 1].reset_index(drop=True)
        
        for f in tqdm([
            ['anchor_first_word'],
            ['anchor_last_word'],
            ['target_first_word'],
            ['target_last_word'],
            ['anchor_first_word', 'target_first_word'],
            ['anchor_last_word', 'target_last_word'],
            ['anchor_first_word', 'anchor_last_word'],
            ['target_first_word', 'target_last_word'],
        ]):
            enc_field = f'{"_".join(f)}_target_mean'
            X[enc_field] = 0
            X_test[enc_field] = 0
            for i in range(FOLD):
                val_idx = X[X['fold'] == i].index
                trn_idx = X[X['fold'] != i].index

                trn_x = X.iloc[trn_idx].reset_index(drop=True)
                val_x = X[f].iloc[val_idx].reset_index(drop=True)
                test_copy = X_test[f].copy()

                trn_x[enc_field] = trn_x.groupby(f)['score'].transform('mean')
                m = trn_x[f + [enc_field]].drop_duplicates(f).reset_index(drop=True)
                val_x = val_x.merge(m, on=f, how='left')
                val_x[enc_field] = val_x[enc_field].fillna(X['score'].mean())
                X.loc[val_idx, enc_field] = val_x[enc_field].values
                test_copy = test_copy.merge(m, on=f, how='left')
                test_copy[enc_field] = test_copy[enc_field].fillna(X['score'].mean())
                X_test[enc_field] += test_copy[enc_field].values / FOLD
        
        return X, X_test
    
    
    X, X_test = feat_eng_core(train_data, test_data)
    cols = [
        f for f in X.columns if f not in ['id', 'anchor', 'target', 'score', 'tag', 'fold', 'bert_proba']
    ]
    X['tree_proba'] = 0
    X_test['tree_proba'] = 0
    clf = LGBMRegressor(
        learning_rate=0.1,
        num_leaves=31,
        n_estimators=3000,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=2022,
        n_jobs=10,
        metric='None'
    )
    for i in range(FOLD):
        print(f'lgbm --------------------- {i} fold ---------------------')
        t = time.time()
        val_idx = X[X['fold'] == i].index
        trn_idx = X[X['fold'] != i].index
        ic(val_idx, trn_idx)
        trn_x = X.iloc[trn_idx].reset_index(drop=True)
        val_x = X.iloc[val_idx].reset_index(drop=True)
        ic(trn_x.shape, val_x.shape)
        clf.fit(
            trn_x[cols], trn_x['score'],
            eval_set=[(val_x[cols], val_x['score'])],
            eval_metric='l1',
            early_stopping_rounds=300,
            verbose=300
        )
        X.loc[val_idx, 'tree_proba'] = clf.predict(val_x[cols])
        X_test['tree_proba'] += clf.predict(X_test[cols]) / FOLD
        print(f'runtime: {time.time() - t}\n')
    
    cols = ['bert_proba', 'tree_proba']
    X['stacking_proba'] = 0
    X_test['stacking_proba'] = 0
    clf = BayesianRidge()
    for i in range(FOLD):
        print(f'ridge --------------------- {i} fold ---------------------')
        t = time.time()
        val_idx = X[X['fold'] == i].index
        trn_idx = X[X['fold'] != i].index
        trn_x = X.iloc[trn_idx].reset_index(drop=True)
        val_x = X.iloc[val_idx].reset_index(drop=True)
        clf.fit(trn_x[cols], trn_x['score'])
        X.loc[val_idx, 'stacking_proba'] += clf.predict(val_x[cols])
        X_test['stacking_proba'] += clf.predict(X_test[cols]) / FOLD
        print(f'runtime: {time.time() - t}\n')
    
    return X[['id', 'score', 'bert_proba', 'tree_proba', 'stacking_proba']], X_test[['id', 'bert_proba', 'tree_proba', 'stacking_proba']]




In [62]:
x_train = pd.DataFrame(x)
ic(x_train.columns)
del x['score']
del x['fold']
x_test = pd.DataFrame(x)
ic(x_test.columns)
a, b = run_stacking(x_train, x_test)

[06/24/22 17:36:09] 1786042895.py:2 in <module>
                    x_train.columns: Index(['id', 'anchor', 'target', 'context', 'fold', 'bert_proba', 'score'], dtype='object')
[06/24/22 17:36:09] 1786042895.py:6 in <module>
                    x_test.columns: Index(['id', 'anchor', 'target', 'context', 'bert_proba'], dtype='object')
100%|██████████| 8/8 [00:01<00:00,  6.21it/s]
[06/24/22 17:37:29] 3007469764.py:317 in run_stacking()
                    val_idx: Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
                                         ...
                                         7183, 7184, 7185, 7186, 7187, 7188, 7189, 7190, 7191, 7192],
                                        dtype='int64', length=7193)
                    trn_idx: Int64Index([ 7193,  7194,  7195,  7196,  7197,  7198,  7199,  7200,  7201,
                                          7202,
                                         ...
                                         36463, 3

lgbm --------------------- 0 fold ---------------------
[300]	valid_0's l1: 0.141532


[06/24/22 17:37:31] 3007469764.py:317 in run_stacking()
                    val_idx: Int64Index([ 7193,  7194,  7195,  7196,  7197,  7198,  7199,  7200,  7201,
                                          7202,
                                         ...
                                         14439, 14440, 14441, 14442, 14443, 14444, 14445, 14446, 14447,
                                         14448],
                                        dtype='int64', length=7256)
                    trn_idx: Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                                             9,
                                         ...
                                         36463, 36464, 36465, 36466, 36467, 36468, 36469, 36470, 36471,
                                         36472],
                                        dtype='int64', length=29217)
[06/24/22 17:37:31] 3007469764.py:320 in run_stacking()
                    trn_x.shape: (29217, 101)
     

runtime: 1.4622762203216553

lgbm --------------------- 1 fold ---------------------
[300]	valid_0's l1: 0.142313
[600]	valid_0's l1: 0.142532


[06/24/22 17:37:33] 3007469764.py:317 in run_stacking()
                    val_idx: Int64Index([14449, 14450, 14451, 14452, 14453, 14454, 14455, 14456, 14457,
                                         14458,
                                         ...
                                         21332, 21333, 21334, 21335, 21336, 21337, 21338, 21339, 21340,
                                         21341],
                                        dtype='int64', length=6893)
                    trn_idx: Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                                             9,
                                         ...
                                         36463, 36464, 36465, 36466, 36467, 36468, 36469, 36470, 36471,
                                         36472],
                                        dtype='int64', length=29580)
[06/24/22 17:37:33] 3007469764.py:320 in run_stacking()
                    trn_x.shape: (29580, 101)
     

runtime: 1.8549184799194336

lgbm --------------------- 2 fold ---------------------
[300]	valid_0's l1: 0.136527


[06/24/22 17:37:34] 3007469764.py:317 in run_stacking()
                    val_idx: Int64Index([21342, 21343, 21344, 21345, 21346, 21347, 21348, 21349, 21350,
                                         21351,
                                         ...
                                         28485, 28486, 28487, 28488, 28489, 28490, 28491, 28492, 28493,
                                         28494],
                                        dtype='int64', length=7153)
                    trn_idx: Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                                             9,
                                         ...
                                         36463, 36464, 36465, 36466, 36467, 36468, 36469, 36470, 36471,
                                         36472],
                                        dtype='int64', length=29320)
[06/24/22 17:37:34] 3007469764.py:320 in run_stacking()
                    trn_x.shape: (29320, 101)
     

runtime: 1.6139953136444092

lgbm --------------------- 3 fold ---------------------
[300]	valid_0's l1: 0.144738


[06/24/22 17:37:36] 3007469764.py:317 in run_stacking()
                    val_idx: Int64Index([28495, 28496, 28497, 28498, 28499, 28500, 28501, 28502, 28503,
                                         28504,
                                         ...
                                         36463, 36464, 36465, 36466, 36467, 36468, 36469, 36470, 36471,
                                         36472],
                                        dtype='int64', length=7978)
                    trn_idx: Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                                             9,
                                         ...
                                         28485, 28486, 28487, 28488, 28489, 28490, 28491, 28492, 28493,
                                         28494],
                                        dtype='int64', length=28495)
[06/24/22 17:37:36] 3007469764.py:320 in run_stacking()
                    trn_x.shape: (28495, 101)
     

runtime: 1.492877721786499

lgbm --------------------- 4 fold ---------------------
[300]	valid_0's l1: 0.141385
runtime: 1.1087117195129395

ridge --------------------- 0 fold ---------------------
runtime: 0.0686333179473877

ridge --------------------- 1 fold ---------------------
runtime: 0.03560924530029297

ridge --------------------- 2 fold ---------------------
runtime: 0.03301715850830078

ridge --------------------- 3 fold ---------------------
runtime: 0.02453446388244629

ridge --------------------- 4 fold ---------------------
runtime: 0.024599075317382812



In [63]:
b

,id,bert_proba,tree_proba,stacking_proba
0,00068e3dfd5515df,0.252783,0.219812,0.241072
1,000c8495ef1763e1,0.023997,0.105973,0.004008
2,0019b063a0fe1dc8,0.015981,0.098650,-0.004335
3,002230e82321e23c,0.475657,0.532071,0.474224
4,0022e4c3050cf263,0.994114,1.029632,1.014073
...,...,...,...,...
36468,ffa2cb9b30cc96be,0.330914,0.059384,0.319842
36469,ffa964af3df23a35,0.533170,0.456617,0.532674
36470,ffb6e883de7d9d67,0.486388,0.426027,0.484119
36471,fff0b4d9fca7fae3,0.386951,0.153019,0.378629


In [64]:
a

,id,score,bert_proba,tree_proba,stacking_proba
0,00068e3dfd5515df,0.25,0.252783,0.238936,0.242448
1,000c8495ef1763e1,0.00,0.023997,0.204718,0.005986
2,0019b063a0fe1dc8,0.00,0.015981,0.218664,-0.002140
3,002230e82321e23c,0.50,0.475657,0.533760,0.475542
4,0022e4c3050cf263,1.00,0.994114,0.997540,1.015442
...,...,...,...,...,...
36468,ffa2cb9b30cc96be,0.00,0.330914,0.201467,0.320080
36469,ffa964af3df23a35,0.50,0.533170,0.431948,0.531520
36470,ffb6e883de7d9d67,0.50,0.486388,0.359960,0.482317
36471,fff0b4d9fca7fae3,0.25,0.386951,0.127218,0.376468


In [65]:
x_train

,id,anchor,target,context,fold,bert_proba,score,tag
0,00068e3dfd5515df,conductor particles,resin cores,B01,0,0.252783,0.25,0
1,000c8495ef1763e1,page file,hidden camera,G09,0,0.023997,0.00,0
2,0019b063a0fe1dc8,predetermined acceleration,predetermined policy,B66,0,0.015981,0.00,0
3,002230e82321e23c,indoor room,air conditioning room,F24,0,0.475657,0.50,0
4,0022e4c3050cf263,tap portion,tap portion,B23,0,0.994114,1.00,0
...,...,...,...,...,...,...,...,...
36468,ffa2cb9b30cc96be,cement composite,organic binders,C04,4,0.330914,0.00,0
36469,ffa964af3df23a35,extend areas,extend area housing,H01,4,0.533170,0.50,0
36470,ffb6e883de7d9d67,water propulsion device,propulsion unit,B63,4,0.486388,0.50,0
36471,fff0b4d9fca7fae3,high gradient magnetic separators,magnetic equipment,C02,4,0.386951,0.25,0


In [66]:
calc_metric(a.score, a.bert_proba)

0.8682403177522899

In [67]:
calc_metric(a.score, a.stacking_proba)

0.8681291501732962